In [1]:
import joblib
import pandas as pd
import random
import json
import re
import spacy
nlp = spacy.load("en_core_web_sm")
import string
import os

In [2]:
new_data_ = joblib.load('/home/gy237/project/Biomedical_datasets/total_pubmed/New_PMID_abstract/abstracts_and_pubdata/abstracts_1253-1575_2.joblib')
print(new_data_.columns)
# old_data = joblib.load('/home/gy237/project/Biomedical_datasets/total_pubmed/New_PMID_abstract/abstracts_and_pubdata/abstracts_1253-1575.joblib')
# print(old_data.columns)

Index(['pmid', 'title', 'abstract', 'journal', 'pub_year', 'pub_month',
       'authors', 'mesh_terms'],
      dtype='object')


In [3]:
print('去重前：', len(new_data_))
new_data = new_data_.drop_duplicates(subset=['pmid'], keep='last')
print('去重复后：', len(new_data))

# print('去重前：', len(old_data))
# old_data = old_data.drop_duplicates(subset=['pmid'], keep='first')
# print('去重复后：', len(old_data))

去重前： 5520745
去重复后： 2888326


In [4]:
# 以data作为keyword进行筛选abstract
new_data = new_data[['pmid', 'abstract']]
print('new_data',len(new_data))
df = new_data[new_data['abstract'].str.contains(' data', case=False)]
print('new_df', len(df))

# old_data = old_data[['pmid', 'abstract']]
# print('old_data',len(old_data))
# old_data = old_data[old_data['abstract'].str.contains(' data', case=False)]
# print('old_data', len(old_data))

new_data 2888326
new_df 760405


In [ ]:
def replace(folder_name):
    file_list = os.listdir(folder_name)
    file_list = [i.split('.')[0] for i in file_list]

    filtered_df = new_data_[new_data_['pmid'].isin(file_list)]

    dic = {'pmid':[], 'abstract':[]}
    for pmid in file_list:
        n_abstract = filtered_df[filtered_df['pmid'] == pmid]['abstract']
        if len(n_abstract) > 0:
            if pmid == '31804926':
                for i in n_abstract:
                    print(str(i))

            n_abstract = n_abstract.iloc[-1]
            n = split_abstracts(str(n_abstract))

            with open(f'{folder_name}/{pmid}.txt', 'r') as file:
                o = file.readlines()

            if len(o) < len(n):
                # print('Old', len(o))
                # print('New', len(n))
                dic['pmid'].append(pmid)
                dic['abstract'].append(n_abstract)
            if len(o) > len(n):
                print(pmid)
                print('Old', len(o))
                print('New', len(n))

    new_df = pd.DataFrame(dic)
    print(len(new_df))
    return new_df

folder_name = '/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_1/agreement'
filtered_df = replace(folder_name)

In [6]:
# 将abstracts分句并进行保存
def ends_with_punctuation(text):
    return text[-1] in string.punctuation if text else False

def split_abstracts(abstract):
    doc = nlp(abstract)
    abstracts = [sent.text for sent in doc.sents]
    assert len(abstracts) != 0
        
    new_abstracts = []
    for j in abstracts:
        j = j.strip().split('\n')   # 有些内部会有\n
        for k in j:
            if len(k.split(' ')) > 5 and ends_with_punctuation(k):
                new_abstracts.append(k)
    return new_abstracts

def save_abstracts(data, name):
    for index, row in data.iterrows():
        pmid = row['pmid']
        abstract = row['abstract']
        doc = split_abstracts(abstract)
        
        # 以pmid为文件名创建txt文件
        with open(f"/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_1/replace/{name}/{pmid}.txt", "w", encoding="utf-8") as file:
            for j in doc:
                file.write(j + '\n')


def replace(folder_name):
    file_list = os.listdir(folder_name)
    file_list = [i.split('.')[0] for i in file_list]

    filtered_df = new_data_[new_data_['pmid'].isin(file_list)]

    dic = {'pmid':[], 'abstract':[]}
    for pmid in file_list:
        n_abstract = filtered_df[filtered_df['pmid'] == pmid]['abstract']
        if len(n_abstract) > 0:
            # if pmid == '31804926':
            #     for i in n_abstract:
            #         print(str(i))

            n_abstract = n_abstract.iloc[-1]
            n = split_abstracts(str(n_abstract))

            with open(f'{folder_name}/{pmid}.txt', 'r') as file:
                o = file.readlines()

            if len(o) != len(n):
                # print('Old', len(o))
                # print('New', len(n))
                dic['pmid'].append(pmid)
                dic['abstract'].append(n_abstract)
            if len(o) > len(n):
                print(pmid)
                print('Old', len(o))
                print('New', len(n))

    new_df = pd.DataFrame(dic)
    print(len(new_df))
    return new_df

    new_df = pd.DataFrame(dic)
    print(len(new_df))
    return new_df

In [7]:
Gui = replace('/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_1/Gui_collections')
Kalpana = replace('/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_1/Kalpana_collections')
agreement = replace('/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_1/agreement')

save_abstracts(Gui, 'Gui_collections')
save_abstracts(Kalpana, 'Kalpana_collections')
save_abstracts(agreement, 'agreement')

0
0
3


In [5]:
import shutil

pmids = os.listdir('/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_2/Batch_2')
print(pmids)

for pmid in pmids:
    with open(f'/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_2/Batch_2/{pmid}', 'r', encoding='utf-8') as f:
        data_2 = f.readlines()
    with open(f'/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_1/Gui_collections/{pmid}', 'r', encoding='utf-8') as f:
        data = f.readlines()

    if len(data_2) != len(data):
        print(pmid)
        assert len(data_2) < len(data)
        destination_folder = '/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_2/replace'
        # 执行复制操作
        shutil.copy(f'/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_1/Gui_collections/{pmid}', destination_folder)

['37591491.txt', '6346023.txt', '22515297.txt', '27016680.txt', '10657818.txt', '30705958.txt', '30895828.txt', '20809428.txt', '34066147.txt', '10713456.txt', '10194149.txt', '12616319.txt', '15778628.txt', '30910098.txt', '37490260.txt', '3000419.txt', '35161585.txt', '33172140.txt', '35356913.txt', '12556355.txt', '26042604.txt', '32152104.txt', '18829980.txt', '22011129.txt', '29035720.txt', '37491428.txt', '39165603.txt', '33569569.txt', '8214108.txt', '31804926.txt', '11169927.txt', '16164819.txt', '30459531.txt', '38228772.txt', '22935033.txt', '10084181.txt', '37590149.txt', '20407350.txt', '19861938.txt', '26142899.txt', '26693134.txt', '21406389.txt', '11308491.txt', '37526980.txt', '37029917.txt', '14871383.txt', '19371988.txt', '31987244.txt', '27042800.txt', '16834830.txt', '31682896.txt', '25689741.txt', '14711821.txt', '27399252.txt', '21903727.txt', '38374749.txt', '37881318.txt', '26190333.txt', '37548437.txt', '25954435.txt', '11932442.txt', '26541884.txt', '33969279.